# CoT - Chain-of-Thought (Agora com memória)

* **Ideia**: Mostrar raciocínio passo-a-passo antes da resposta final 
* **Quando**: tarefas que exigem lógica (datas, contagens, regras de negócio).

In [1]:
# Imports necessários
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage
from langgraph.graph import START, MessagesState, StateGraph

from IPython.display import display, Markdown
from pathlib import Path
import os
from dotenv import load_dotenv

In [2]:
dotenv_path = Path("../.env")
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [4]:
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

## **PROMPT 01:** Aqui vou fazer um prompt CoT para o analista de QA

In [5]:
prompt = """
Você é um **engenheiro de QA experiente**, responsável por validar os requisitos de uma tela de login.

Antes de criar o plano de testes, **pense passo a passo sobre cada requisito**, identificando possíveis cenários de uso, limites, exceções e validações.

**Requisitos da tela de login**:

* O campo de nome de usuário deve aceitar apenas e-mails válidos.
* A senha deve ter entre 8 e 20 caracteres, contendo letras maiúsculas, minúsculas, números e caracteres especiais.
* O sistema deve permitir no máximo 3 tentativas de login. Após isso, um captcha deve ser exibido.
* O link "lembrar senha" deve levar o usuário para a tela correspondente.

Siga esta cadeia de pensamento:

1. Para cada requisito, identifique **o objetivo da regra**.
2. Em seguida, pense em **cenários válidos** e **cenários inválidos**.
3. Considere também **casos limite e exceções possíveis**.
4. Por fim, **gere os testes com base nesse raciocínio**, no formato Gherkin (Given–When–Then), conforme exemplo abaixo:

```
**Teste 1**: Verificação de acesso com credenciais válidas  
**Cenário**: O usuário entra com dados corretos

**Dado** que o usuário está na tela de login  
**Quando** ele digita o e-mail e a senha corretos  
**Então** o sistema deve redirecioná-lo para a página inicial
```

Comece pensando em voz alta sobre o primeiro requisito e continue com os demais. Depois disso, apresente os testes estruturados.
"""

display(Markdown(f"*Prompt:*"))
display(Markdown(f"----"))
display(Markdown(f"{prompt}"))
display(Markdown(f"----"))

input_messages = [HumanMessage(prompt)]
output = app.invoke({"messages": input_messages}, config)
resposta = output["messages"][-1].content

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{resposta}"))
display(Markdown(f"----"))

*Prompt:*

----


Você é um **engenheiro de QA experiente**, responsável por validar os requisitos de uma tela de login.

Antes de criar o plano de testes, **pense passo a passo sobre cada requisito**, identificando possíveis cenários de uso, limites, exceções e validações.

**Requisitos da tela de login**:

* O campo de nome de usuário deve aceitar apenas e-mails válidos.
* A senha deve ter entre 8 e 20 caracteres, contendo letras maiúsculas, minúsculas, números e caracteres especiais.
* O sistema deve permitir no máximo 3 tentativas de login. Após isso, um captcha deve ser exibido.
* O link "lembrar senha" deve levar o usuário para a tela correspondente.

Siga esta cadeia de pensamento:

1. Para cada requisito, identifique **o objetivo da regra**.
2. Em seguida, pense em **cenários válidos** e **cenários inválidos**.
3. Considere também **casos limite e exceções possíveis**.
4. Por fim, **gere os testes com base nesse raciocínio**, no formato Gherkin (Given–When–Then), conforme exemplo abaixo:

```
**Teste 1**: Verificação de acesso com credenciais válidas  
**Cenário**: O usuário entra com dados corretos

**Dado** que o usuário está na tela de login  
**Quando** ele digita o e-mail e a senha corretos  
**Então** o sistema deve redirecioná-lo para a página inicial
```

Comece pensando em voz alta sobre o primeiro requisito e continue com os demais. Depois disso, apresente os testes estruturados.


----

*Resposta:*

----

Vamos lá! Vou analisar cada requisito da tela de login passo a passo.

### Requisito 1: O campo de nome de usuário deve aceitar apenas e-mails válidos.

**Objetivo da regra:** Garantir que o usuário forneça um e-mail no formato correto para evitar problemas de comunicação e autenticação.

**Cenários válidos:**
- O usuário informa um e-mail como "usuario@dominio.com".
- O usuário informa um e-mail com subdomínio, como "usuario@subdominio.dominio.com".
- O usuário fornece um e-mail com caracteres válidos, como "user.name@dominio.co.uk".

**Cenários inválidos:**
- O usuário informa um e-mail sem o símbolo "@", como "usuariodominio.com".
- O usuário informa um e-mail com mais de um símbolo "@", como "usuario@@dominio.com".
- O usuário informa um e-mail sem domínio, como "usuario@".
- O usuário fornece um e-mail com caracteres inválidos, como "usuario.com".

**Casos limite e exceções possíveis:**
- O uso de caracteres especiais válidos, como "usuario+filter@dominio.com".
- O limite de caracteres (muito longo ou muito curto).

### Requisito 2: A senha deve ter entre 8 e 20 caracteres, contendo letras maiúsculas, minúsculas, números e caracteres especiais.

**Objetivo da regra:** Garantir a segurança das senhas, exigindo complexidade.

**Cenários válidos:**
- A senha é "Senha123!".
- A senha é "Complex@2023".
- A senha contém 8 a 20 caracteres, misturando todos esses elementos.

**Cenários inválidos:**
- A senha tem menos de 8 caracteres, como "Abc1!".
- A senha tem mais de 20 caracteres, como "UmaSenhaMuitoLonga123!@#".
- A senha não contém letras maiúsculas, como "senha123!".
- A senha não contém números, como "SenhaAberta!!".
- A senha não contém caracteres especiais, como "Senha123".

**Casos limite e exceções possíveis:**
- Testar senhas exatamente com 8 e 20 caracteres.
- Testar senhas com repetição de caracteres.

### Requisito 3: O sistema deve permitir no máximo 3 tentativas de login. Após isso, um captcha deve ser exibido.

**Objetivo da regra:** Evitar tentativas de acesso não autorizadas ao limitar as tentativas de login.

**Cenários válidos:**
- O usuário faz 3 tentativas e na última, usa uma senha correta e consegue acessar.
- O usuário faz 3 tentativas com senhas incorretas e o captcha é exibido.

**Cenários inválidos:**
- O usuário tenta 4 vezes seguidas e o captcha não é exibido.

**Casos limite e exceções possíveis:**
- O sistema deve contar apenas tentativas falhas.
- Se o usuário consegue entrar após o captcha, as tentativas devem ser resetadas.

### Requisito 4: O link "lembrar senha" deve levar o usuário para a tela correspondente.

**Objetivo da regra:** Fornecer um meio para o usuário recuperar ou redefinir sua senha, caso a tenha esquecido.

**Cenários válidos:**
- O usuário clica no link "lembrar senha" e é redirecionado para a página de recuperação.

**Cenários inválidos:**
- O link "lembrar senha" não funciona ou não redireciona corretamente.
- O link não é visível ou está quebrado.

**Casos limite e exceções possíveis:**
- Testar com usuários que estejam logados – o sistema deve redirecionar de forma apropriada.

### Testes Estruturados (Formato Gherkin)

```gherkin
**Teste 1**: Verificação de e-mail válido
**Cenário**: O usuário informa um e-mail válido
  **Dado** que o usuário está na tela de login  
  **Quando** ele digita "usuario@dominio.com" no campo de nome de usuário  
  **Então** o sistema deve aceitar o e-mail como válido

**Teste 2**: Verificação de e-mail inválido
**Cenário**: O usuário informa um e-mail inválido
  **Dado** que o usuário está na tela de login  
  **Quando** ele digita "usuariodominio.com" no campo de nome de usuário  
  **Então** o sistema deve rejeitar o e-mail como inválido

**Teste 3**: Verificação de senha válida
**Cenário**: O usuário informa uma senha válida
  **Dado** que o usuário está na tela de login  
  **Quando** ele digita "Senha123!" no campo de senha  
  **Então** o sistema deve aceitar a senha como válida

**Teste 4**: Verificação de senha inválida
**Cenário**: O usuário informa uma senha inválida
  **Dado** que o usuário está na tela de login  
  **Quando** ele digita "senha" no campo de senha  
  **Então** o sistema deve rejeitar a senha como inválida

**Teste 5**: Limite de tentativas de login
**Cenário**: O usuário ultrapassa o limite de tentativas
  **Dado** que o usuário está na tela de login  
  **Quando** ele tenta logar 4 vezes com senhas incorretas  
  **Então** o sistema deve exibir um captcha

**Teste 6**: Redirecionamento do link "lembrar senha"
**Cenário**: O usuário clica no link "lembrar senha"
  **Dado** que o usuário está na tela de login  
  **Quando** ele clica no link "lembrar senha"  
  **Então** o sistema deve redirecionar o usuário para a tela de recuperação de senha
```

Esses testes cobrem a maioria dos cenários relacionados aos requisitos apresentados e garantem que todos os aspectos da tela de login foram considerados.

----

## Vamos testar a memória

In [6]:
prompt = """
Adicione mais casos de testes, por favor.
"""

display(Markdown(f"*Prompt:*"))
display(Markdown(f"----"))
display(Markdown(f"{prompt}"))
display(Markdown(f"----"))

input_messages = [HumanMessage(prompt)]
output = app.invoke({"messages": input_messages}, config)
resposta = output["messages"][-1].content

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{resposta}"))
display(Markdown(f"----"))

*Prompt:*

----


Adicione mais casos de testes, por favor.


----

*Resposta:*

----

Claro! Vamos expandir os casos de teste para cada um dos requisitos, considerando mais cenários para garantir uma cobertura mais abrangente.

### Requisito 1: O campo de nome de usuário deve aceitar apenas e-mails válidos.

### Novos Cenários Válidos:
- O usuário informa um e-mail que contém números, como "usuario123@dominio.com".
- O usuário informa um e-mail com caracteres especiais no nome, como "user.name@example.com".

### Novos Cenários Inválidos:
- O usuário informa um e-mail com um domínio inválido, como "usuario@dominio..com".
- O usuário informa um e-mail com um espaço em branco no início ou no final, como " usuario@dominio.com ".
- O usuário informa um e-mail apenas com caracteres especiais ou vazio, como "@dominio.com" ou "".

### Requisito 2: A senha deve ter entre 8 e 20 caracteres, contendo letras maiúsculas, minúsculas, números e caracteres especiais.

### Novos Cenários Válidos:
- A senha contém exatamente 20 caracteres, como "Senha!123456789012345".
- A senha é composta apenas de letras e números, como "Senha123".

### Novos Cenários Inválidos:
- A senha tem exatamente 8 caracteres, mas não contém caracteres especiais, como "Senha12".
- A senha tem mais de 20 caracteres e é composta apenas de letras, como "AAAAAAAAAAAAAAAAAAAAA".
- A senha não contém letras minúsculas, como "SENHA123!".
- A senha apenas com caracteres especiais, como "@#&*()!".
  
### Requisito 3: O sistema deve permitir no máximo 3 tentativas de login. Após isso, um captcha deve ser exibido.

### Novos Cenários Válidos:
- O usuário faz 2 tentativas com senhas incorretas seguidas de uma tentativa correta e consegue acessar.
  
### Novos Cenários Inválidos:
- O usuário tenta logar com a senha correta após 3 tentativas incorretas e deve ser bloqueado, mostrando o captcha.
- O sistema não deve permitir que o usuário faça mais tentativas após o captcha ser mostrado até que o captcha seja resolvido.

### Requisito 4: O link "lembrar senha" deve levar o usuário para a tela correspondente.

### Novos Cenários Válidos:
- O usuário já está logado e clica no link "lembrar senha" e é redirecionado para a tela de troca de senha e recebe uma mensagem informando que ele não pode redefinir a senha enquanto estiver logado.

### Novos Cenários Inválidos:
- O link "lembrar senha" é clicado e não redireciona devido a um erro de sistema (ex: 404).
- O link "lembrar senha" está visível somente quando o campo de senha não é preenchido.

### Testes Estruturados (Formato Gherkin - Adicionando novos casos)

```gherkin
**Teste 1**: Verificação de e-mail válido
**Cenário**: O usuário informa um e-mail válido
  **Dado** que o usuário está na tela de login  
  **Quando** ele digita "usuario@dominio.com" no campo de nome de usuário  
  **Então** o sistema deve aceitar o e-mail como válido

**Teste 2**: Verificação de e-mail inválido
**Cenário**: O usuário informa um e-mail inválido
  **Dado** que o usuário está na tela de login  
  **Quando** ele digita "usuariodominio.com" no campo de nome de usuário  
  **Então** o sistema deve rejeitar o e-mail como inválido

**Teste 3**: Verificação de e-mail com espaço em branco
**Cenário**: O usuário informa um e-mail com espaço em branco
  **Dado** que o usuário está na tela de login  
  **Quando** ele digita " usuario@dominio.com " no campo de nome de usuário  
  **Então** o sistema deve rejeitar o e-mail como inválido

**Teste 4**: Verificação de senha válida
**Cenário**: O usuário informa uma senha válida
  **Dado** que o usuário está na tela de login  
  **Quando** ele digita "Senha123!" no campo de senha  
  **Então** o sistema deve aceitar a senha como válida

**Teste 5**: Verificação de senha inválida
**Cenário**: O usuário informa uma senha inválida
  **Dado** que o usuário está na tela de login  
  **Quando** ele digita "senha" no campo de senha  
  **Então** o sistema deve rejeitar a senha como inválida

**Teste 6**: Limite de tentativas de login
**Cenário**: O usuário ultrapassa o limite de tentativas
  **Dado** que o usuário está na tela de login  
  **Quando** ele tenta logar 4 vezes com senhas incorretas  
  **Então** o sistema deve exibir um captcha

**Teste 7**: O usuário informa um e-mail que contém caracteres especiais
**Cenário**: O usuário informa um e-mail com caracter especial
  **Dado** que o usuário está na tela de login  
  **Quando** ele digita "user.name@dominio.com" no campo de nome de usuário  
  **Então** o sistema deve aceitar o e-mail como válido

**Teste 8**: O usuário informa uma senha exatamente com 8 caracteres, mas sem caracteres especiais
**Cenário**: O usuário informa uma senha com 8 caracteres, mas sem caracteres especiais
  **Dado** que o usuário está na tela de login  
  **Quando** ele digita "Senha12" no campo de senha  
  **Então** o sistema deve rejeitar a senha como inválida

**Teste 9**: O usuário clica no link "lembrar senha"
**Cenário**: O usuário clica no link "lembrar senha"
  **Dado** que o usuário está na tela de login  
  **Quando** ele clica no link "lembrar senha"  
  **Então** o sistema deve redirecionar o usuário para a tela de recuperação de senha

**Teste 10**: O usuário já está logado e clica no link "lembrar senha"
**Cenário**: O usuário já está logado e tentado acessar "lembrar senha"
  **Dado** que o usuário já está logado  
  **Quando** ele clica no link "lembrar senha"  
  **Então** o sistema deve mostrar uma mensagem informando que ele não pode redefinir a senha enquanto estiver logado
```

Esses novos casos de teste aumentam a cobertura, permitindo que cenários extras e potenciais falhas sejam identificados durante o processo de QA da tela de login.

----

## Pedindo para alterar o formato

In [7]:
prompt = """
Agora coloque no formato tabela, conforme exemplo abaixo:

| Nº do Teste | Título                     | Pré-condições        | Ação                                             | Resultado Esperado                         |
| ----------- | -------------------------- | -------------------- | ------------------------------------------------ | ------------------------------------------ |
| 1           | E-mail inválido não aceito | Tela de login aberta | Inserir "usuariosemarroba.com" e clicar em Login | Mensagem de erro "E-mail inválido" exibida |
| 2           | Senha curta bloqueada      | Tela de login aberta | Inserir "abc123" como senha e clicar em Login    | Mensagem de erro "Senha inválida" exibida  |
"""

display(Markdown(f"*Prompt:*"))
display(Markdown(f"----"))
display(Markdown(f"{prompt}"))
display(Markdown(f"----"))

input_messages = [HumanMessage(prompt)]
output = app.invoke({"messages": input_messages}, config)
resposta = output["messages"][-1].content

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{resposta}"))
display(Markdown(f"----"))

*Prompt:*

----


Agora coloque no formato tabela, conforme exemplo abaixo:

| Nº do Teste | Título                     | Pré-condições        | Ação                                             | Resultado Esperado                         |
| ----------- | -------------------------- | -------------------- | ------------------------------------------------ | ------------------------------------------ |
| 1           | E-mail inválido não aceito | Tela de login aberta | Inserir "usuariosemarroba.com" e clicar em Login | Mensagem de erro "E-mail inválido" exibida |
| 2           | Senha curta bloqueada      | Tela de login aberta | Inserir "abc123" como senha e clicar em Login    | Mensagem de erro "Senha inválida" exibida  |


----

*Resposta:*

----

Claro! Aqui está a tabela com os casos de teste formatados conforme solicitado:

| Nº do Teste | Título                                       | Pré-condições        | Ação                                                     | Resultado Esperado                                   |
|-------------|----------------------------------------------|----------------------|----------------------------------------------------------|-----------------------------------------------------|
| 1           | E-mail válido aceito                          | Tela de login aberta | Inserir "usuario@dominio.com" e clicar em Login         | Sistema aceita o e-mail como válido                 |
| 2           | E-mail inválido não aceito                   | Tela de login aberta | Inserir "usuariodominio.com" e clicar em Login          | Mensagem de erro "E-mail inválido" exibida          |
| 3           | E-mail com espaço em branco                   | Tela de login aberta | Inserir " usuario@dominio.com " e clicar em Login       | Mensagem de erro "E-mail inválido" exibida          |
| 4           | Senha válida aceita                           | Tela de login aberta | Inserir "Senha123!" como senha e clicar em Login        | Sistema aceita a senha como válida                  |
| 5           | Senha curta bloqueada                         | Tela de login aberta | Inserir "senha" como senha e clicar em Login            | Mensagem de erro "Senha inválida" exibida           |
| 6           | Senha sem caracteres especiais                | Tela de login aberta | Inserir "Senha12" como senha e clicar em Login          | Mensagem de erro "Senha inválida" exibida           |
| 7           | Tentativas de login ultrapassadas             | Tela de login aberta | Inserir senhas incorretas 4 vezes e clicar em Login     | Captcha é exibido                                   |
| 8           | E-mail com caracteres especiais válidos       | Tela de login aberta | Inserir "user.name@dominio.com" e clicar em Login       | Sistema aceita o e-mail como válido                 |
| 9           | Redirecionamento do link "lembrar senha"     | Tela de login aberta | Clicar no link "lembrar senha"                          | Sistema redireciona para a tela de recuperação      |
| 10          | E-mail com domínio inválido                   | Tela de login aberta | Inserir "usuario@dominio..com" e clicar em Login        | Mensagem de erro "E-mail inválido" exibida          |
| 11          | Senha longa bloqueada                         | Tela de login aberta | Inserir "UmaSenhaMuitoLonga123!@#" e clicar em Login     | Mensagem de erro "Senha inválida" exibida           |
| 12          | Senha sem letras minúsculas                  | Tela de login aberta | Inserir "SENHA123!" como senha e clicar em Login        | Mensagem de erro "Senha inválida" exibida           |
| 13          | Senha apenas com caracteres especiais         | Tela de login aberta | Inserir "@#&*()!" como senha e clicar em Login          | Mensagem de erro "Senha inválida" exibida           |
| 14          | O usuário clica no link "lembrar senha"      | Tela de login aberta | Clicar no link "lembrar senha"                          | Sistema redireciona para a tela de recuperação      |
| 15          | Usuário logado clica no link "lembrar senha" | Usuário está logado  | Clicar no link "lembrar senha"                          | Mensagem informando que não pode redefinir exibida  |

Esses casos de teste na tabela possibilitam uma visão clara das condições necessárias, ações a serem realizadas e resultados esperados, facilitando o processo de validação da tela de login.

----